In [13]:
import requests
import json
import os
from dotenv import load_dotenv
import asyncio
from typing import Optional, Type, Any, Dict, Union, Tuple, List
from pydantic import BaseModel
import logging

load_dotenv()  # This loads the .env file into the environment

import openai

client = openai.AsyncOpenAI()

async def call_openai(
    messages,
    model_name = "o3-mini",
    output_type: Optional[Type[BaseModel]] = None,
    max_tokens: int = 15000,
    reasoning_effort = "medium",
    max_retries=3,
    tools: Optional[Dict[str, Any]] = None,
    tool_choice: Optional[str]=None,
) -> Union[str, BaseModel]:
    """
    Calls the LLM using OpenAI's chat completion or HF, returning either raw text
    or a Pydantic-validated object.
    """

    last_exception = None
    for attempt in range(max_retries):
        try:
            if output_type is None:
                response = await client.chat.completions.create(
                    messages=messages,
                    model=model_name,
                    max_completion_tokens=max_tokens,
                    tools=tools,
                    tool_choice=tool_choice,
                )
                if tools:
                    result = response
                else:
                    result = response.choices[0].message.content
                # logging.info("\nRESPONSE:", result)
                # logging.info("=" * 80)
                return result
            else:
                response = await client.beta.chat.completions.parse(
                    messages=messages,
                    model=model_name,
                    max_completion_tokens=max_tokens,
                    response_format=output_type,
                )
                parsed_obj = response.choices[0].message.parsed
                result = output_type.model_validate(parsed_obj)
                # logging.info("\nRESPONSE:", result)
                # logging.info("=" * 80)
                return result

        except Exception as e:
            logging.info(
                f"[DEBUG] Attempt {attempt+1}/{max_retries} failed with exception: {e}"
            )
            logging.info("[DEBUG] User messages (verbatim):")
            for idx, msg in enumerate(messages):
                logging.info(f"  Message {idx+1} - role='{msg['role']}':")
                logging.info(msg["content"])
            last_exception = e
            await asyncio.sleep(1.0 * (attempt + 1))

    raise last_exception


await call_openai(messages=[
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What is the capital of the United States?"},
])

'The capital of the United States is Washington, D.C. It serves as the center of the U.S. federal government, hosting the White House, the U.S. Capitol, the Supreme Court, and many federal agencies. Unlike most U.S. cities, Washington, D.C. is not part of any state but is a federal district established to serve as the nation’s capital.'

In [19]:
propose_new_tool = {
    "type": "function",
    "function": {
        "name": "propose_new_tool",
        "description": "Propose the function specs for one or two new python functions you can call anytime as new tool calls later on. Brainstorm some functions that do specific algorithmic computations about the board that helps your decision, and write detailed specs of those.",
        "parameters": {
            "type": "object",
            "properties": {
                "tool_name_1": {
                    "type": "string",
                    "description": "Name of the tool you will use"
                },
                "tool_name_2": {
                    "type": "string",
                    "description": "Name of the tool you will use"
                },
                "function_spec_1": {
                    "type": "string",
                    "description": "A detailed description of the function you want to propose, as well as the signature of its outputs. The function should only receive an input which is a 2D array representing the current visible board state, where in the array X represents a hit, O represents a miss, and ~ represents unknown cells."  
                },
                "function_spec_2": {
                    "type": "string",
                    "description": "A detailed description of the function you want to propose, as well as the signature of its outputs. The function should only receive an input which is a 2D array representing the current visible board state, where in the array X represents a hit, O represents a miss, and ~ represents unknown cells."  
                }
            },
            "required": ["tool_name_1", "function_spec_1", "tool_name_2", "function_spec_2"],
            "additionalProperties": False
        },
        "strict": True
    }
}

In [20]:
import chess
import chess.svg
import random

class ChessGame:
    def __init__(self):
        self.board = chess.Board()
        self.game_over = False
        self.current_player = chess.WHITE  # White starts
        self.tools = {"White": [], "Black": [propose_new_tool]}
        self.move_history = []
        
    def make_move(self, move_str):
        """
        Make a move using algebraic notation (e.g., "e2e4", "g1f3").
        
        Args:
            move_str (str): The move in UCI format
            
        Returns:
            bool: True if the move was successful, False otherwise
        """
        try:
            # Parse the move string
            move = chess.Move.from_uci(move_str)
            
            # Check if the move is legal
            if move in self.board.legal_moves:
                # Make the move
                self.board.push(move)
                self.move_history.append(move_str)
                
                # Switch player
                self.current_player = not self.current_player
                
                # Check if the game is over
                self.check_game_state()
                return True
            else:
                print(f"Illegal move: {move_str}")
                return False
        except ValueError:
            print(f"Invalid move format: {move_str}")
            return False
    
    def make_move_san(self, san_move):
        """
        Make a move using Standard Algebraic Notation (e.g., "e4", "Nf3").
        
        Args:
            san_move (str): The move in SAN format
            
        Returns:
            bool: True if the move was successful, False otherwise
        """
        try:
            # Parse and make the move
            move = self.board.parse_san(san_move)
            self.board.push(move)
            self.move_history.append(self.board.uci(move))
            
            # Switch player
            self.current_player = not self.current_player
            
            # Check if the game is over
            self.check_game_state()
            return True
        except ValueError:
            print(f"Invalid or illegal move: {san_move}")
            return False
    
    def get_legal_moves(self):
        """
        Get all legal moves for the current position.
        
        Returns:
            list: List of legal moves in UCI format
        """
        return [move.uci() for move in self.board.legal_moves]
    
    def is_check(self):
        """Check if the current player is in check."""
        return self.board.is_check()
    
    def is_checkmate(self):
        """Check if the current position is checkmate."""
        return self.board.is_checkmate()
    
    def is_stalemate(self):
        """Check if the current position is stalemate."""
        return self.board.is_stalemate()
    
    def is_insufficient_material(self):
        """Check if there is insufficient material to checkmate."""
        return self.board.is_insufficient_material()
    
    def is_game_over(self):
        """Check if the game is over for any reason."""
        return self.game_over
    
    def check_game_state(self):
        """Check the current state of the game and update game_over if needed."""
        if self.board.is_checkmate():
            self.game_over = True
            winner = "Black" if self.current_player else "White"
            print(f"Checkmate! {winner} wins.")
        elif self.board.is_stalemate():
            self.game_over = True
            print("Game over: Stalemate.")
        elif self.board.is_insufficient_material():
            self.game_over = True
            print("Game over: Insufficient material to checkmate.")
        elif self.board.is_seventyfive_moves():
            self.game_over = True
            print("Game over: 75-move rule.")
        elif self.board.is_fivefold_repetition():
            self.game_over = True
            print("Game over: Fivefold repetition.")
    
    def get_board_fen(self):
        """Get the current board position in FEN notation."""
        return self.board.fen()
    
    def get_current_player(self):
        """Get the current player (True for white, False for black)."""
        return "White" if self.current_player else "Black"
    
    def get_move_history(self):
        """Get the move history."""
        return self.move_history
    
    def export_pgn(self):
        """Export the game in PGN format."""
        game = chess.pgn.Game.from_board(self.board)
        game.headers["Event"] = "Python Chess Game"
        game.headers["Site"] = "Python Chess"
        game.headers["Date"] = "????.??.??"
        game.headers["Round"] = "1"
        game.headers["White"] = "Player 1"
        game.headers["Black"] = "Player 2"
        game.headers["Result"] = self.board.result()
        return str(game)
    
    def reset(self):
        """Reset the game to the starting position."""
        self.board = chess.Board()
        self.game_over = False
        self.current_player = chess.WHITE
        self.move_history = []
    
    def get_board_ascii(self):
        """Get a string representation of the board."""
        return str(self.board)
    
    def make_prompt(self):
        player = self.get_current_player()
        system = f"You are a master at chess."
        user_prompt = f"The current board state is:\n{self.get_board_ascii()}\n"
        user_prompt += f"You are playing as {player}. Use the function calling tools at your disposal to make the best move possible. Your final output should be ONLY a legal move in UCI format (e.g., e2e4)."
        return [{"role": "system", "content": system}, {"role": "user", "content": user_prompt}]

    async def llm_make_move(self):
        messages = self.make_prompt()

        move = await call_openai(messages=messages, tools=self.tools[self.get_current_player()])
        print(move)

        self.make_move(move)


# Initialize the game
game = ChessGame()
await game.llm_make_move()
await game.llm_make_move()

e2e4
ChatCompletion(id='chatcmpl-B6Nh76xMJ0jCtFr6zQe6sr4xvRkjD', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bKteUWpYn7bzH6VjccCJ0c3B', function=Function(arguments='{"tool_name_1": "analyze_board", "tool_name_2": "choose_best_move", "function_spec_1": "This function takes a 2D array representing the chess board state (with symbols: \'r\', \'n\', \'b\', \'q\', \'k\', etc. for pieces, and \'.\' for empty squares) and returns an analysis object containing piece positions, material counts, and available moves. The function should output a dictionary with keys \'pieces\' (a mapping from piece symbols to their board coordinates), \'material\' (a mapping from color to total piece values), and \'moves\' (a mapping from each piece to a list of legal moves in UCI format).", "function_spec_2": "This function takes a 

TypeError: object of type 'ChatCompletion' has no len()

In [ ]:


while not game.is_game_over():
    # Get the current player
    player = game.get_current_player()


    
    # Get the legal moves
    legal_moves = game.get_legal_moves()
    
    # Print the board
    print(game.get_board_ascii())
    
    # Print the current player and legal moves
    print(f"Current player: {player}")
    print(f"Legal moves: {legal_moves}")
    
    # Let the AI make a move
    if player == "White":
        move = random.choice(legal_moves)
        print(f"AI plays: {move}")
        game.make_move(move)
    else:
        # Ask the user for a move
        move = input("Enter your move (e.g., e2e4): ")
        game.make_move(move)

# Make moves (with UCI notation)
game.make_move("e2e4")  # Move pawn from e2 to e4

# Check the board state
print(game.get_board_ascii())
print(f"Current player: {game.get_current_player()}")
print(f"Legal moves: {game.get_legal_moves()}")

# Check if the game is over
if game.is_game_over():
    print("Game is over")